<a href="https://colab.research.google.com/github/KU81-ComSci-ProjectEnd-GameCard/main/blob/master/Predictor/data/json_unzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data
**https://drive.google.com/drive/folders/1c7MwTdLxnPgvmPbBEfNWa45YAUU53H0l** make sure to load data form this before processing.  

raw_data file so large to push to github

In [1]:
import re
def clear_SearingBlowPlus(lst):
    return [re.sub(r'Searing Blow\+\d+', 'Searing Blow', item) for item in lst]

In [2]:
def upgrade_lst(lst):
  new_lst = list()
  for item in lst:
    if(item == 'Searing Blow'):
      new_lst.append('Searing Blow')
    else:
      new_lst.append(item+'+1')
  return new_lst

In [3]:
def get_ALL_IC_CARD():
  IC_ATTACK_CARDS = ["Strike_R", "Bash", "Anger", "Body Slam", "Clash", "Cleave", "Clothesline", "Headbutt", "Heavy Blade", "Iron Wave", "Perfected Strike", "Pommel Strike", "Sword Boomerang", "Thunderclap", "Twin Strike", "Wild Strike", "Blood for Blood", "Carnage", "Dropkick", "Hemokinesis", "Pummel", "Rampage", "Reckless Charge", "Searing Blow", "Sever Soul", "Uppercut", "Whirlwind", "Bludgeon", "Feed", "Fiend Fire", "Immolate", "Reaper"]
  IC_SKILL_CARDS = ["Defend_R", "Armaments", "Flex", "Havoc", "Shrug It Off", "True Grit", "Warcry", "Battle Trance", "Bloodletting", "Burning Pact", "Disarm", "Dual Wield", "Entrench", "Flame Barrier", "Ghostly Armor", "Infernal Blade", "Intimidate", "Power Through", "Rage", "Second Wind", "Seeing Red", "Sentinel", "Shockwave", "Spot Weakness", "Double Tap", "Exhume", "Impervious", "Limit Break", "Offering"]
  IC_POWER_CARDS = ["Combust", "Dark Embrace", "Evolve", "Feel No Pain", "Fire Breathing", "Inflame", "Metallicize", "Rupture", "Barricade", "Berserk", "Brutality", "Corruption", "Demon Form", "Juggernaut"]
  IRONCLAD_CARDS = IC_ATTACK_CARDS + IC_SKILL_CARDS + IC_POWER_CARDS
  return IRONCLAD_CARDS + upgrade_lst(IRONCLAD_CARDS)

In [4]:
def get_ALL_COLORLESS_CARD():
    COLORLESS_CARDS = ["Dramatic Entrance", "Flash of Steel", "Mind Blast", "Swift Strike", "HandOfGreed", "Bite", "Expunger", "Ritual Dagger", "Shiv", "Smite", "Through Violence", "Bandage Up", "Blind", "Dark Shackles", "Deep Breath", "Discovery", "Enlightenment", "Finesse", "Forethought", "Good Instincts", "Impatience", "Jack Of All Trades", "Madness", "Panacea", "PanicButton", "Purity", "Trip", "Apotheosis", "Chrysalis", "Master of Strategy", "Metamorphosis", "Secret Technique", "Secret Weapon", "The Bomb", "Thinking Ahead", "Transmutation", "Violence", "Apparition", "Beta", "Insight", "J.A.X.", "Miracle", "Safety", "Magnetism", "Mayhem", "Panache", "Sadistic Nature"]
    CURSE_CARDS = ["AscendersBane", "Clumsy", "Curse of the Bell", "Decay", "Doubt", "Injury", "Necronomicurse", "Normality", "Pain", "Parasite", "Pride", "Regret", "Shame", "Writhe"]
    return COLORLESS_CARDS + upgrade_lst(COLORLESS_CARDS) + CURSE_CARDS 

In [5]:
def get_ALL_TS_CARD():
    TS_ALL_CARD =[ 	
        "Strike_G" , "Defend_G" ,"Neutralize" , "Survivor",
        "Bane", "Dagger Spray", "Dagger Throw", "Flying Knee", "Poisoned Stab", "Quick Slash", "Slice", "Sneaky Strike", "Sucker Punch",
        "All-Out Attack", "Backstab", "Choke", "Dash", "Endless Agony", "Eviscerate", "Finisher", "Flechettes", "Heel Hook", "Masterful Stab", "Predator", "Riddle with Holes", "Skewer",
        "Die Die Die", "Glass Knife", "Grand Finale", "Unload",
        "Acrobatics", "Backflip", "Blade Dance", "Cloak and Dagger", "Deadly Poison", "Deflect", "Dodge and Roll", "Outmaneuver", "Piercing Wail", "Prepared",
        "Blur", "Bouncing Flask", "Calculated Gamble", "Catalyst", "Concentrate", "Crippling Cloud", "Distraction", "Escape Plan", "Expertise", "Leg Sweep", "Reflex", "Setup", "Tactician", "Terror",
        "Adrenaline", "Alchemize", "Bullet Time", "Burst", "Corpse Explosion", "Doppelganger", "Malaise", "Nightmare", "Phantasmal Killer", "Storm of Steel",
        "Accuracy", "Caltrops", "Footwork", "Infinite Blades", "Noxious Fumes", "Well Laid Plans",
        "A Thousand Cuts", "After Image", "Envenom", "Tools of the Trade", "Wraith Form"
    ]
    return TS_ALL_CARD + upgrade_lst(TS_ALL_CARD)
    

In [64]:
import pandas as pd
def get_ic_run(json_data):
  df = pd.DataFrame(json_data)
  runInfo = df['event'].apply(pd.Series)
  runInfo['master_deck'] = runInfo['master_deck'].apply(clear_SearingBlowPlus)
  ## รับแค่ run ของ ironclad
  ALL_CARD = get_ALL_TS_CARD() + get_ALL_COLORLESS_CARD()
  ic_run = runInfo[runInfo['master_deck'].apply(lambda x: all(word in ALL_CARD for word in x))]
  ## เกณคือ ต้องผ่านชั้น 50 แล้ว
  return ic_run.loc[ic_run.floor_reached >= 50]

In [70]:
import os
import gzip
import json

def get_quality_run(directory, run_num,start=0, end=None):
    quality_run = pd.DataFrame()
    index = 0 
    try:
        counter = 0
        for file_name in os.listdir(directory):
            file_path = os.path.join(directory, file_name)
            if os.path.isfile(file_path) and file_name.endswith('.json.gz'):
                if counter >= start and (end is None or counter < end):
                    with gzip.open(file_path, 'rt', encoding='utf-8') as gz_file:
                        run_info = json.load(gz_file)
                        ic_run = get_ic_run(run_info)
                        quality_run = pd.concat([quality_run, ic_run], axis=0, join='outer')
                        quality_run.drop_duplicates(subset=['play_id'])
                        if(len(quality_run)>index):
                            print(counter, '   :     ',len(quality_run))
                            index = index + 2000
                        if(len(quality_run) >= run_num):
                            break
            counter += 1
    except Exception as e:
        print(f"Error: {e}")
    return quality_run
                        

In [71]:
raw_data = 'Monthly_2020_11'

In [72]:
quality_run = get_quality_run(raw_data, 8000)

0    :      2
602    :      2001
1208    :      4003


In [73]:
quality_run = quality_run.reset_index()

In [74]:
len(quality_run)

5289

In [75]:
import os
file_path = os.path.join('processed_data', 'TS_quality_run5k.json')
quality_run.to_json(file_path)

In [76]:
(quality_run.loc[quality_run.ascension_level >= 10]).iloc[0].master_deck

['AscendersBane',
 'Strike_G',
 'Strike_G',
 'Strike_G',
 'Strike_G',
 'Strike_G',
 'Defend_G',
 'Defend_G',
 'Defend_G',
 'Defend_G',
 'Defend_G',
 'Survivor+1',
 'Neutralize+1',
 'Footwork+1',
 'Noxious Fumes+1',
 'Dagger Throw',
 'Dash+1',
 'Backflip+1',
 'Dagger Spray+1',
 'Adrenaline',
 'Backflip+1',
 'Dodge and Roll+1',
 'Blur',
 'Terror+1',
 'Terror+1',
 'Terror+1']